### The University of Melbourne, School of Computing and Information Systems
# COMP90086 Computer Vision, 2022 Semester 2

## Final Project


**Group Members:**  
  Yizhou Wang : 873236 \
  Victor Lee: 921250

### 1. Disparity Map
To calculate a disparity map for the left image using classical(non deep learning) methods. You should create an image of floating point numbers that estimate the disparity (x-x') for every pixel in the left image.





In [1]:
## Edit this when different group member is using this
dirpath =  './Dataset'

import os
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import threading
import math
import time

In [2]:
def readImgPair(img_name):
  imgL = np.int16(cv2.imread(os.path.join(dirpath, img_name + "-left.jpg"), cv2.IMREAD_GRAYSCALE))
  imgR = np.int16(cv2.imread(os.path.join(dirpath, img_name + "-right.jpg"), cv2.IMREAD_GRAYSCALE))
  ground_truth = cv2.imread(os.path.join(dirpath, img_name + "-disparity.png"), -1)
  return [imgL, imgR, ground_truth]

# display the images
def displayImgPair(imgL, imgR):
  plt.subplots(figsize=(15,15))
  plt.subplot(1,2,1)
  plt.imshow(imgL, cmap='gray')  
  plt.title('image L')
  plt.axis('off')
  plt.subplot(1,2,2)
  plt.imshow(imgR, cmap='gray')  
  plt.title('image R')
  plt.axis('off')
  plt.show()

# read from the directory, obatin all image pairs
allName = []
for file_entry in os.listdir(dirpath):
  if file_entry[-4:] == ".jpg" and file_entry[-8:-4] == "left":
    name = file_entry[:-9]
    if name not in allName:
      allName.append(name)

print(allName)



['2018-07-09-16-11-56_2018-07-09-16-11-56-702', '2018-07-09-16-11-56_2018-07-09-16-13-38-366', '2018-07-09-16-11-56_2018-07-09-16-14-54-205', '2018-07-09-16-11-56_2018-07-09-16-16-13-272', '2018-07-09-16-11-56_2018-07-09-16-25-10-570', '2018-07-09-16-11-56_2018-07-09-16-26-00-067', '2018-07-09-16-11-56_2018-07-09-16-27-53-156', '2018-07-09-16-11-56_2018-07-09-16-29-12-297', '2018-07-09-16-11-56_2018-07-09-16-29-56-824', '2018-07-09-16-11-56_2018-07-09-16-30-47-660', '2018-07-09-16-11-56_2018-07-09-16-32-28-044', '2018-07-09-16-11-56_2018-07-09-16-33-15-274', '2018-07-09-16-11-56_2018-07-09-16-35-14-342', '2018-07-09-16-11-56_2018-07-09-16-37-13-816', '2018-07-09-16-11-56_2018-07-09-16-38-04-951', '2018-07-09-16-11-56_2018-07-09-16-38-52-780', '2018-07-09-16-11-56_2018-07-09-16-40-37-947', '2018-07-09-16-11-56_2018-07-09-16-41-23-475', '2018-07-09-16-11-56_2018-07-09-16-42-22-318', '2018-07-09-16-11-56_2018-07-09-16-46-01-859', '2018-07-09-16-11-56_2018-07-09-16-46-51-712', '2018-07-09-

In [143]:
def GaussianFilter(img):
  """Returns a 2D Gaussian kernel array."""
  gau_kernel = cv2.getGaussianKernel(15,5)
  gau_kern2d = np.outer(gau_kernel, gau_kernel)
  return cv2.filter2D(img,-1,gau_kern2d)

## algorithm 1: Window based (SAD)
## Cost estimation
# SAD Function
# Input windows are np arrays
def SAD(windowL, windowR):
  return np.sum(abs(windowL-windowR))
# SSD Function
def SSD(windowL, windowR):
  return np.sum((windowL-windowR)**2)

# CT Function
def CT(windowL, windowR):
  height, width = windowL.shape
  centreL = windowL[height//2, width//2]
  centreR = windowR[height//2, width//2]

  bit_windowL = np.copy(windowL)
  bit_windowR = np.copy(windowR)

  bit_windowL[bit_windowL > centreL] = centreL
  bit_windowL = bit_windowL // centreL

  bit_windowR[bit_windowR > centreR] = centreR
  bit_windowR = bit_windowR // centreR

  return np.sum(abs(bit_windowL - bit_windowR))

def SADKernel(windowL, windowR, kernel):
  return np.sum(abs(windowL-windowR) * kernel)

# CT Function
def CTKernel(windowL, windowR, kernel):
  height, width = windowL.shape
  centreL = windowL[height//2, width//2]
  centreR = windowR[height//2, width//2]

  bit_windowL = np.copy(windowL)
  bit_windowR = np.copy(windowR)

  bit_windowL[bit_windowL > centreL] = centreL
  bit_windowL = bit_windowL // centreL

  bit_windowR[bit_windowR > centreR] = centreR
  bit_windowR = bit_windowR // centreR

  return np.sum(abs(bit_windowL - bit_windowR) * kernel)


def CTAD(windowL_CT, windowR_CT, windowL_AD, windowR_AD, lambda_CT, lambda_AD):
  CT_cost = CT(windowL_CT, windowR_CT)
  SAD_cost = SAD(windowL_AD, windowR_AD)
  CT_rho = 1 - math.exp(-CT_cost / lambda_AD)
  SAD_rho = 1 - math.exp(-SAD_cost / lambda_CT)
  return CT_rho + SAD_rho

## Cost Aggregation
def findMinCostIndex(windowL, row, column, imgR, windowSize, searchThreshold, costFunction):
  halfWindowSize = windowSize//2
  lowerRange = max(halfWindowSize, column - searchThreshold)
  minX = None
  minCost = None 
  for x in range(lowerRange, column):
    windowR = imgR[row-halfWindowSize:row+halfWindowSize+1, x-halfWindowSize:x+halfWindowSize+1]
    currCost = costFunction(windowL, windowR)
    
    if minCost is None or currCost < minCost:
      minCost = currCost
      minX = x
    
  return minX

def weightedWindow(window_size, std):
  """Returns a 2D Gaussian kernel array."""
  gau_kernel = cv2.getGaussianKernel(window_size, std)
  gau_kern2d = np.outer(gau_kernel, gau_kernel)
  return gau_kern2d/gau_kern2d.sum()

# Fixed Window
def fixedWindow(imgL, imgR, windowSize, min_searchThreshold, max_searchThreshold, costFunction): # could include search range

  height, width = imgL.shape
  disparityMap = np.zeros((height, width))
  halfWindowSize = windowSize // 2

  maxPixels = width * max_searchThreshold
  minPixels = width * min_searchThreshold
  gradient = (maxPixels - minPixels) / (height - windowSize)
  for y in range(halfWindowSize, height-halfWindowSize):
    searchThreshold = int(gradient * y + minPixels)
    for x in range(windowSize, width - halfWindowSize):
      windowL = imgL[y-halfWindowSize:y+halfWindowSize+1, x-halfWindowSize:x+halfWindowSize+1]
      index = findMinCostIndex(windowL, y, x, imgR, windowSize, searchThreshold, costFunction)
      disparityMap[y,x] = x-index
  disparity = np.array(disparityMap)
  return disparity

def fixedWindowASW(imgL, imgR, windowSize, min_searchThreshold, max_searchThreshold, costFunction, std): # could include search range

  height, width = imgL.shape
  disparityMap = np.zeros((height, width))
  halfWindowSize = windowSize // 2

  kernel = weightedWindow(windowSize, std)
  maxPixels = width * max_searchThreshold
  minPixels = width * min_searchThreshold
  gradient = (maxPixels - minPixels) / (height - windowSize)
  for y in range(halfWindowSize, height-halfWindowSize):
    searchThreshold = int(gradient * y + minPixels)
    for x in range(windowSize, width - halfWindowSize):
      windowL = imgL[y-halfWindowSize:y+halfWindowSize+1, x-halfWindowSize:x+halfWindowSize+1]
      index = findMinCostIndexASW(windowL, y, x, imgR, windowSize, searchThreshold, costFunction, kernel)
      disparityMap[y,x] = x-index
  disparity = np.array(disparityMap)
  return disparity

def findMinCostIndexASW(windowL, row, column, imgR, windowSize, searchThreshold, costFunction, kernel):
  halfWindowSize = windowSize//2
  lowerRange = max(halfWindowSize, column - searchThreshold)
  minX = None
  minCost = None 
  for x in range(lowerRange, column):
    windowR = imgR[row-halfWindowSize:row+halfWindowSize+1, x-halfWindowSize:x+halfWindowSize+1]
    currCost = costFunction(windowL, windowR, kernel)
    
    if minCost is None or currCost < minCost:
      minCost = currCost
      minX = x
  return minX
## combine
# SAD small window no smoothing
# CT larger winddow + smoothing  

def multiWindowCTAD(imgL, imgR, windowSize1, windowSize2, lambda_CT, lambda_AD, min_searchThreshold, max_searchThreshold):
  height, width = imgL.shape
  disparityMap = np.zeros((height, width))
  halfWindowSize1 = windowSize1 // 2
  halfWindowSize2 = windowSize2 // 2

  maxPixels = width * max_searchThreshold
  minPixels = width * min_searchThreshold
  gradient = (maxPixels - minPixels) / (height - windowSize2)
  for y in range(halfWindowSize2, height-halfWindowSize2):
    searchThreshold = int(gradient * y + minPixels)
    for x in range(windowSize2, width- halfWindowSize2):
      if (halfWindowSize1 <= y < height - halfWindowSize1 and halfWindowSize1 < x < width - halfWindowSize1):
        windowL_CT = imgL[y-halfWindowSize1:y+halfWindowSize1+1, x-halfWindowSize1:x+halfWindowSize1+1]
        windowL_AD = imgL[y-halfWindowSize2:y+halfWindowSize2+1, x-halfWindowSize2:x+halfWindowSize2+1]
        index = findMinCostIndexCTAD(windowL_CT, windowL_AD, lambda_CT, lambda_AD, y, x, imgR, windowSize1, windowSize2, searchThreshold)
        disparityMap[y,x] = x-index
      else:
        windowL_AD = imgL[y-halfWindowSize2:y+halfWindowSize2+1, x-halfWindowSize2:x+halfWindowSize2+1]
        index = findMinCostIndex(windowL_AD, y, x, imgR, windowSize2, searchThreshold, SAD)
        disparityMap[y,x] = x-index
  disparity = np.array(disparityMap)
  return disparity

def findMinCostIndexCTAD(windowL_CT, windowL_AD, lambda_CT, lambda_AD, row, column, imgR, windowSize1, windowSize2, searchThreshold):
  halfWindowSize1 = windowSize1//2
  halfWindowSize2 = windowSize2//2
  lowerRange = max(halfWindowSize1, column - searchThreshold)
  minX = None
  minCost = None 
  for x in range(lowerRange, column):
    windowR_CT = imgR[row-halfWindowSize1:row+halfWindowSize1+1, x-halfWindowSize1:x+halfWindowSize1+1]
    windowR_AD = imgR[row-halfWindowSize2:row+halfWindowSize2+1, x-halfWindowSize2:x+halfWindowSize2+1]
    currCost = CTAD(windowL_CT, windowR_CT, windowL_AD, windowR_AD, lambda_CT, lambda_AD)
    
    if minCost is None or currCost < minCost:
      minCost = currCost
      minX = x
    
  return minX

### 2. Statistics

a. The rms (root mean squared) error between the values in your disparity map and those in the ground truth\
b. The fractions of pixels with errors less than 4, 2, 1, 0.5 and 0.25 pixels\
c. The runtime of your algorithm in seconds per image

---



In [78]:

from numpy import mean, sqrt, square, arange


def getErrorRate(disparity, ground_truth, window_size):
  ground_truth = ground_truth/256
  ground_truth = ground_truth[window_size//2:-window_size//2, window_size: -window_size//2]
  ground_truth_dot = np.copy(ground_truth)
  ground_truth_dot[ground_truth_dot > 0] = 1
  ground_truth_count = np.sum(ground_truth_dot > 0)

  disparity = disparity[window_size//2:-window_size//2, window_size: -window_size//2]
  disparity_dot = disparity * ground_truth_dot
  diff = disparity_dot - ground_truth

  rms = sqrt(np.sum(square(diff))/ground_truth_count)
  threshold = {4:0, 2:0, 1:0, 0.5:0, 0.25:0}
  for key in threshold:

    count = ground_truth_count - len(diff[diff >= key])
    threshold[key] = count
  
  return rms, threshold, diff, ground_truth_count
  
  

In [100]:
def testImages(imgPaths, windowSize, minSearch, maxSearch, costFunction, std=None):
    imgLs = []
    imgRs= []
    groundTruths = []
    for imgPath in imgPaths:
        imgL, imgR, gt = readImgPair(imgPath)
        imgLs.append(imgL)
        imgRs.append(imgR)
        groundTruths.append(gt)
    
    runTimes = []
    diffs = []
    rmsList = []
    thresholds = []
    thresholds_rates = []
    count_total = 0
    diff_sum = 0
    for i in range(len(imgLs)):
        print(f'image {i} starts.')
        startTime = time.time()
        if std is None:
          disparity = fixedWindow(imgLs[i], imgRs[i], windowSize, minSearch, maxSearch, costFunction)
        else:
          disparity = fixedWindowASW(imgLs[i], imgRs[i], windowSize, minSearch, maxSearch, costFunction, std)
        duration = time.time()-startTime
        runTimes.append(duration)

        rms, threshold, diff, ground_truth_count = getErrorRate(disparity, groundTruths[i], windowSize)
        count_total += ground_truth_count
        rmsList.append(rms)
        thresholds.append(threshold)
        thresholds_rate = {}
        for key in threshold:
          thresholds_rate[key] = threshold[key] / ground_truth_count
        thresholds_rates.append(thresholds_rate)
        diffs.append(diff)
        print(f'image {i} done.')

    # overall threshold
    threshold_all = {4:0, 2:0, 1:0, 0.5:0, 0.25:0}
    for i in range(len(runTimes)):
      for key in threshold_all:
        threshold_all[key] += thresholds[i][key]
    for key in threshold_all:
      threshold_all[key] = threshold_all[key] / count_total

    # overall runtime
    runTimesSum = sum(runTimes)

    # overall rms
    for i in range(len(runTimes)):
      diff_sum += np.sum(square(diffs[i]))
    
    overall_rms = sqrt(diff_sum/count_total)


    
    return runTimes, rmsList, thresholds_rates, overall_rms, threshold_all






In [96]:
bs_runTimes, bs_rmsList, bs_thresholds_rates, bs_overall_rms, bs_threshold_overall = testImages(allName[0:5], 1, 1, 1, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [97]:
print(bs_runTimes, bs_rmsList, bs_thresholds_rates, bs_overall_rms, bs_threshold_overall)

[577.6938161849976, 1209.2039630413055, 574.4336378574371, 574.256513595581, 4990.5103397369385] [338.0222339835602, 330.1988663027823, 360.78253582613615, 256.38325829649244, 346.13608690241557] [{4: 0.24389411148794757, 2: 0.22528623130676256, 1: 0.2119022147360754, 0.5: 0.20338925181526415, 0.25: 0.19933762775134864}, {4: 0.20593803594894983, 2: 0.18968973428068692, 1: 0.17751742878143306, 0.5: 0.16943225605274295, 0.25: 0.1651335278526405}, {4: 0.21993561224261665, 2: 0.20452055713295625, 1: 0.1927608486664133, 0.5: 0.1858861141541282, 0.25: 0.18223076751101075}, {4: 0.26485213294948967, 2: 0.2402637055869194, 1: 0.2226791789733428, 0.5: 0.2133821861641804, 0.25: 0.2083847409678344}, {4: 0.2141601896517155, 2: 0.19646879828275562, 1: 0.18345972849611378, 0.5: 0.17514477455270266, 0.25: 0.17029733331760766}] 329.4600837743498 {4: 0.22910544121662585, 2: 0.21071813692594632, 1: 0.1972157557862389, 0.5: 0.18902407982487401, 0.25: 0.18467138420811485}


In [85]:
SAD_5_10 = testImages(allName[0:5], 21, 0.05, 0.10, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [81]:
SAD_2_5_10 = testImages(allName[0:5], 21, 0.025, 0.10, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [82]:
SAD_2_10 = testImages(allName[0:5], 21, 0.02, 0.10, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [83]:
SAD_2_5_5 = testImages(allName[0:5], 21, 0.02, 0.05, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [84]:
SAD_2_5_15 = testImages(allName[0:5], 21, 0.02, 0.15, SAD)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.


In [86]:
print(SAD_5_10)

([115.76127529144287, 115.56549429893494, 116.86851668357849, 117.79547810554504, 118.41525721549988], [16.193742337507533, 9.13732408081882, 16.04044689972401, 12.480170832312426, 14.687678738275551], [{4: 0.9540459660043094, 2: 0.8892506583672493, 1: 0.8190926502274359, 0.5: 0.7397055302848935, 0.25: 0.6914891070146038}, {4: 0.9650031878513882, 2: 0.8903031356865473, 1: 0.793566336289341, 0.5: 0.6763809192604184, 0.25: 0.6031182982669681}, {4: 0.9595250002907687, 2: 0.9159794833622164, 1: 0.8460205398992777, 0.5: 0.7816210935228369, 0.25: 0.7248979401946987}, {4: 0.9272732024094181, 2: 0.85595756079077, 1: 0.770765552114774, 0.5: 0.6838095984738609, 0.25: 0.6231429579397123}, {4: 0.8788909298515614, 2: 0.8108151715236063, 1: 0.7452216650945738, 0.5: 0.6853869830710421, 0.25: 0.6189246884773867}], 13.966002145626181, {4: 0.9378477024091221, 2: 0.8735592374706035, 1: 0.7959935963418739, 0.5: 0.7142428110218485, 0.25: 0.6531537531879028})


In [87]:
print(SAD_2_5_10)

([101.6704580783844, 101.43501305580139, 100.97102189064026, 100.2474091053009, 100.43426895141602], [16.08794715638411, 9.050688129435315, 15.970785207829607, 12.630751868793519, 14.516731100817905], [{4: 0.9543332535312425, 2: 0.8894900646396936, 1: 0.8192961455590136, 0.5: 0.7398970553028489, 0.25: 0.691668661718937}, {4: 0.9655132440734945, 2: 0.8908015997217875, 1: 0.794018431577117, 0.5: 0.6768793832956588, 0.25: 0.6039181591607257}, {4: 0.9603158910896847, 2: 0.9165726514614033, 1: 0.8464043545516928, 0.5: 0.7819467544400377, 0.25: 0.7251770781237279}, {4: 0.9297688578783009, 2: 0.8598643722315864, 1: 0.7759528569375302, 0.5: 0.6899638064625717, 0.25: 0.6304861954973672}, {4: 0.8794122027503352, 2: 0.8112371543464231, 1: 0.745569180360423, 0.5: 0.6857344983368913, 0.25: 0.6192597924837413}], 13.902923564462974, {4: 0.9387414050749717, 2: 0.874641852895088, 1: 0.797238482982109, 0.5: 0.7156620786700542, 0.25: 0.6548418582233966})


In [88]:
print(SAD_2_10)

([97.28573322296143, 96.74161195755005, 97.05008912086487, 96.98687076568604, 96.79761791229248], [16.060992572180556, 9.049466096792306, 15.9600851120331, 12.686433828624294, 14.485389760119938], [{4: 0.9544649269810869, 2: 0.8895858271486713, 1: 0.8193799377543691, 0.5: 0.7399688771845823, 0.25: 0.6917404836006703}, {4: 0.9655480206340926, 2: 0.8908247840955196, 1: 0.7940416159508491, 0.5: 0.6770068973511854, 0.25: 0.6043702544485017}, {4: 0.9606299212598425, 2: 0.9168517893904325, 1: 0.8466486002395934, 0.5: 0.7821677386338525, 0.25: 0.7253515393293711}, {4: 0.9301739119072818, 2: 0.8601779624475716, 1: 0.776318712189513, 0.5: 0.6904211255275502, 0.25: 0.6309304483033462}, {4: 0.8795487266047759, 2: 0.8113488556818746, 1: 0.7456684704363798, 0.5: 0.6858213771533536, 0.25: 0.6193466713002036}], 13.8969100773691, {4: 0.9389424276258268, 2: 0.8748041241108385, 1: 0.7973983322394155, 0.5: 0.715850991428689, 0.25: 0.6550864760262444})


In [89]:
print(SAD_2_5_5)

([57.13777017593384, 56.9134886264801, 56.48231911659241, 57.05947971343994, 56.46379637718201], [16.710585084413374, 13.224093403207487, 16.860848865558154, 16.89705812321584, 14.70632048726232], [{4: 0.9877902801053388, 2: 0.9355757720852287, 1: 0.8768972947091214, 0.5: 0.8040339956906871, 0.25: 0.757840555422552}, {4: 0.9826696806352518, 2: 0.9640410363415058, 1: 0.9262852837187735, 0.5: 0.8676172259896829, 0.25: 0.83371007940648}, {4: 0.9849614440735528, 2: 0.9572918968585352, 1: 0.8998825294548669, 0.5: 0.8421358703869549, 0.25: 0.7882738808313658}, {4: 0.9694118876824377, 2: 0.9319639893901976, 1: 0.88233833771053, 0.5: 0.8200253485424588, 0.25: 0.772490298302693}, {4: 0.9473886710023334, 2: 0.8964776845554286, 1: 0.8424887057538599, 0.5: 0.7906096410663754, 0.25: 0.7287767462642109}], 15.727767484035667, {4: 0.9748406956833434, 2: 0.9377459801544725, 1: 0.8862963169277942, 0.5: 0.8255971944033385, 0.25: 0.7770732569770568})


In [90]:
print(SAD_2_5_15)

([137.6831831932068, 134.70630645751953, 134.04342222213745, 136.54215216636658, 133.8995246887207], [16.901447644283856, 9.3669620709395, 16.28356054517456, 13.29473153744577, 16.04826163652832], [{4: 0.9511970313622217, 2: 0.8866531003112281, 1: 0.8166267656212592, 0.5: 0.7373593488149389, 0.25: 0.6891908067991381}, {4: 0.9630904770184895, 2: 0.8892366544948704, 1: 0.7928708050773778, 0.5: 0.6758708630383122, 0.25: 0.6026314264185939}, {4: 0.9566987287593482, 2: 0.9134556112539108, 1: 0.843822328708173, 0.5: 0.7796206050314611, 0.25: 0.7229323439444516}, {4: 0.9208838017587184, 2: 0.8502476056080384, 1: 0.7658395724720055, 0.5: 0.6793017391190728, 0.25: 0.6191054839089021}, {4: 0.8743235863575436, 2: 0.8069180360422976, 1: 0.7416348111006305, 0.5: 0.6818373628555826, 0.25: 0.6154123020404111}], 14.64119600011989, {4: 0.9342074988677345, 2: 0.8704663965375682, 1: 0.7932785809261085, 0.5: 0.7117167083647179, 0.25: 0.650743904536086})


In [91]:
SAD_11 = testImages(allName[0:5], 11, 0.02, 0.10, SAD)
print(SAD_11)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([88.50709438323975, 87.56518769264221, 86.82421350479126, 86.59972143173218, 86.9639413356781], [17.015652654366068, 11.915201892880125, 17.05723406443202, 14.4889693939031, 15.630435473139906], [{4: 0.9317209128756516, 2: 0.8924094179659713, 1: 0.8230108103695584, 0.5: 0.7396881669018437, 0.25: 0.6863593427481895}, {4: 0.9266113303397288, 2: 0.8887150179162698, 1: 0.8070712568603439, 0.5: 0.6913639043860843, 0.25: 0.6188256905701456}, {4: 0.9478649227544091, 2: 0.9224354008111927, 1: 0.8528574032721141, 0.5: 0.7802032538850658, 0.25: 0.7199904297498063}, {4: 0.8901160494770767, 2: 0.8399638221169157, 1: 0.7526018776830868, 0.5: 0.6682972191437052, 0.25: 0.6109858473140469}, {4: 0.8666626389560174, 2: 0.8231391976800386, 1: 0.7591227646205897, 0.5: 0.6925205413243113, 0.25: 0.626087481875302}], 15.344013054874218, {4: 0.9135545615312598,

In [92]:
SAD_31 = testImages(allName[0:5], 31, 0.02, 0.10, SAD)
print(SAD_31)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([113.01340913772583, 109.80523610115051, 109.8106255531311, 109.88193106651306, 109.81733655929565], [14.694972336348672, 6.747917616927881, 14.835967127168244, 11.411080454868086, 13.384827539442654], [{4: 0.9327909009219709, 2: 0.8585531579660015, 1: 0.7976218318787774, 0.5: 0.7289109910264774, 0.25: 0.687182264675833}, {4: 0.9519638718878127, 2: 0.8523976469190089, 1: 0.7574781626953473, 0.5: 0.6451363717392596, 0.25: 0.5768851387485887}, {4: 0.9537925540137219, 2: 0.8988929713789701, 1: 0.8318886074745241, 0.5: 0.7724943221679211, 0.25: 0.7216375937882733}, {4: 0.9260338345864662, 2: 0.8605397422126746, 1: 0.7936761546723953, 0.5: 0.7089822771213748, 0.25: 0.649422663802363}, {4: 0.8669123930261329, 2: 0.7891259708954558, 1: 0.7292843751195685, 0.5: 0.6740214521662606, 0.25: 0.6135421582256686}], 12.575319389522258, {4: 0.92710192111

In [93]:
CT_11 = testImages(allName[0:5], 11, 0.02, 0.10, CT)
print(CT_11)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([195.82657289505005, 195.40195679664612, 196.75095629692078, 195.53619861602783, 195.21925258636475], [15.907625398112742, 14.409965301941353, 14.947523693351512, 16.34152682434462, 16.20717279730062], [{4: 0.7341138878846893, 2: 0.6721321034157036, 1: 0.5993749344030973, 0.5: 0.5271775256265233, 0.25: 0.48665321687210644}, {4: 0.7771692293736109, 2: 0.7212205742277861, 1: 0.6443166870776069, 0.5: 0.546775071438291, 0.25: 0.4949312831677779}, {4: 0.7656655881146607, 2: 0.7121405459599872, 1: 0.6273640796609397, 0.5: 0.5537301189445382, 0.25: 0.5035660575126464}, {4: 0.7619393383523777, 2: 0.6969847517866015, 1: 0.6174825798397473, 0.5: 0.5492541496286671, 0.25: 0.5064903631800869}, {4: 0.708675688738521, 2: 0.661829386176897, 1: 0.6063315611406477, 0.5: 0.5482962783953601, 0.25: 0.49855002416626387}], 15.553700074144096, {4: 0.7498250475

In [94]:
CT_21 = testImages(allName[0:5], 21, 0.02, 0.10, CT)
print(CT_21)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([216.314710855484, 216.4140820503235, 218.82084107398987, 218.05232501029968, 216.77716445922852], [13.080921872816687, 10.815269533939267, 12.415903696502761, 13.228855672382746, 13.86530825466131], [{4: 0.8292674168063203, 2: 0.747019391908068, 1: 0.6598515681110845, 0.5: 0.5775077807038544, 0.25: 0.5304764184821642}, {4: 0.872729380397612, 2: 0.7995710890859561, 1: 0.7064858285515563, 0.5: 0.5946328174810178, 0.25: 0.5344693676462065}, {4: 0.8488933344188697, 2: 0.7736307703043767, 1: 0.6761883715791065, 0.5: 0.596215354912246, 0.25: 0.5385152188325056}, {4: 0.8427214404243921, 2: 0.7519501391556583, 1: 0.6527511008323207, 0.5: 0.5709171207191669, 0.25: 0.5202723008375472}, {4: 0.7865263366926476, 2: 0.7238122424663654, 1: 0.6551779774611528, 0.5: 0.5897830511840342, 0.25: 0.5339323834582733}], 12.693073044045917, {4: 0.83658804182237

In [95]:
CT_31 = testImages(allName[0:5], 31, 0.02, 0.10, CT)
print(CT_31)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([266.59801840782166, 891.1766865253448, 268.94236421585083, 268.00841903686523, 265.6540229320526], [11.266943083186586, 8.492989984934583, 10.543963856159627, 11.17267265128843, 12.080961919532857], [{4: 0.8538756016199116, 2: 0.759955193933948, 1: 0.6694567879959132, 0.5: 0.5889782001255555, 0.25: 0.5422764928174891}, {4: 0.8947768732545012, 2: 0.8081407094895716, 1: 0.7110226394913542, 0.5: 0.5995127458553687, 0.25: 0.5382137976112663}, {4: 0.8652183735835147, 2: 0.7783921330812494, 1: 0.6787595571885516, 0.5: 0.5988061689199634, 0.25: 0.5427056207564894}, {4: 0.8625134264232008, 2: 0.7574516648764769, 1: 0.6530880773361977, 0.5: 0.5673469387755102, 0.25: 0.5169172932330827}, {4: 0.8019692119326081, 2: 0.7253178925351053, 1: 0.6508347468975985, 0.5: 0.5851390819697221, 0.25: 0.5296083257872384}], 10.74697140330911, {4: 0.8562840171981

In [112]:
SAD_21_STD2 = testImages(allName[0:5], 21, 0.02, 0.10, SADKernel, 2)
print(SAD_21_STD2)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([118.80636739730835, 118.54531598091125, 118.8469831943512, 119.86183571815491, 120.57525038719177], [16.725781901306707, 12.911058579725646, 16.966380045190274, 15.005412584257472, 15.592662918956977], [{4: 0.9045606894900646, 2: 0.8672971031841035, 1: 0.7988747905195116, 0.5: 0.7173090734977257, 0.25: 0.6668422312664591}, {4: 0.8987654320987655, 2: 0.8652640120558743, 1: 0.7932185706833594, 0.5: 0.6835448907436388, 0.25: 0.61773604590506}, {4: 0.9233184847462752, 2: 0.8970213656823178, 1: 0.8388676304679049, 0.5: 0.7706881913025274, 0.25: 0.7146977750380907}, {4: 0.8588452040296343, 2: 0.8131394300497824, 1: 0.7354082552624358, 0.5: 0.658787712490037, 0.25: 0.6047064664915788}, {4: 0.8472670406592862, 2: 0.8077495904284366, 1: 0.7479149084049049, 0.5: 0.6816139601846796, 0.25: 0.6199796455344289}], 15.509804029113386, {4: 0.88760175252

In [113]:
SAD_21_STD3 = testImages(allName[0:5], 21, 0.02, 0.10, SADKernel, 3)
print(SAD_21_STD3)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([119.84654140472412, 117.92231774330139, 117.84680485725403, 119.01057600975037, 119.06256771087646], [16.503169799577428, 11.440276269808583, 16.64340397002939, 14.061295047386082, 15.224062034047218], [{4: 0.9298898731146756, 2: 0.8893344505626047, 1: 0.818817333014125, 0.5: 0.7335887000239406, 0.25: 0.6786689011252095}, {4: 0.9296817944705269, 2: 0.8918217121660001, 1: 0.8104677447400452, 0.5: 0.6914623543731525, 0.25: 0.6175853474758013}, {4: 0.9481850219239581, 2: 0.923446422963747, 1: 0.8569999651077589, 0.5: 0.7829237371916398, 0.25: 0.7207573942474325}, {4: 0.8885578769942378, 2: 0.8402388512145088, 1: 0.7522898618896424, 0.5: 0.6665882691126703, 0.25: 0.6059738936145193}, {4: 0.864580747654272, 2: 0.8211909844611031, 1: 0.7561808072283175, 0.5: 0.6883160403117708, 0.25: 0.6201409919078588}], 14.90002038816608, {4: 0.913250292451

In [114]:
SAD_21_STD5 = testImages(allName[0:5], 21, 0.02, 0.10, SADKernel, 5)
print(SAD_21_STD5)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([118.9973840713501, 118.86689376831055, 119.16393733024597, 118.90335893630981, 119.3147656917572], [16.181528675829394, 9.864093362730467, 16.288257833008633, 13.077071985021226, 14.854069601106703], [{4: 0.9489226717740005, 2: 0.8982643045247786, 1: 0.8257242039741441, 0.5: 0.7417644242279148, 0.25: 0.6878381613598277}, {4: 0.9558337680403408, 2: 0.9040166927490871, 1: 0.8126122993102649, 0.5: 0.6896771575957804, 0.25: 0.612867327421318}, {4: 0.9599088149431838, 2: 0.9318205608346224, 1: 0.8592214377929495, 0.5: 0.7866339454983193, 0.25: 0.7243629258307261}, {4: 0.9197601034847713, 2: 0.8608704741745391, 1: 0.7694981249918336, 0.5: 0.6782956371761201, 0.25: 0.6161394431160414}, {4: 0.8799210643896143, 2: 0.825497691505734, 1: 0.7587375266842079, 0.5: 0.6919525393436926, 0.25: 0.6219406245345778}], 14.25866913195262, {4: 0.9337836561400

In [118]:
SAD_21_STD10 = testImages(allName[0:5], 21, 0.02, 0.10, SADKernel, 10)
print(SAD_21_STD10)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([119.71490216255188, 118.22961282730103, 119.0016222000122, 118.99831700325012, 118.32181882858276], [16.115846271060033, 9.1972457440233, 16.03692457582555, 12.767515545161702, 14.586712845739985], [{4: 0.9548719176442423, 2: 0.8944577447929136, 1: 0.8232224084271008, 0.5: 0.7418960976777591, 0.25: 0.6914172851328705}, {4: 0.9647365675534689, 2: 0.8976062134121602, 1: 0.8023532139338087, 0.5: 0.6825247782994261, 0.25: 0.6076972120790587}, {4: 0.9613975505646728, 2: 0.923341746240361, 1: 0.8516730829621186, 0.5: 0.7836332127612556, 0.25: 0.7251538166296421}, {4: 0.9302000444252806, 2: 0.8643722315863746, 1: 0.7763971097435093, 0.5: 0.6885134517136399, 0.25: 0.627467889668509}, {4: 0.8819937447252147, 2: 0.8186218537457182, 1: 0.751638286253289, 0.5: 0.6891972397358884, 0.25: 0.6208236111800626}], 13.982630832846082, {4: 0.939497056109511

In [115]:
CT_31_STD2 = testImages(allName[0:5], 31, 0.02, 0.10, CTKernel, 2)
print(CT_31_STD2)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([302.0222246646881, 298.55588722229004, 316.4607446193695, 300.97391510009766, 297.83916449546814], [17.42154157283136, 16.147167565273488, 16.72510585825821, 18.122613340992316, 17.164939594502776], [{4: 0.7017565455015449, 2: 0.6501310946712786, 1: 0.5918093526508205, 0.5: 0.5346939277933013, 0.25: 0.5023695515700587}, {4: 0.7400796244577812, 2: 0.6954423911105829, 1: 0.6333828510309585, 0.5: 0.555493493374532, 0.25: 0.5145403767306436}, {4: 0.7249551124270205, 2: 0.6805074971164936, 1: 0.6159407365129193, 0.5: 0.5596737178801174, 0.25: 0.5190549233641304}, {4: 0.72328141783029, 2: 0.6711063372717508, 1: 0.6092239527389903, 0.5: 0.5547529538131042, 0.25: 0.5211331901181525}, {4: 0.6948741821521037, 2: 0.6531049523639471, 1: 0.6050097567819199, 0.5: 0.557067608759422, 0.25: 0.5153493948244416}], 17.10195175714674, {4: 0.7172627183935184

In [116]:
CT_31_STD3 = testImages(allName[0:5], 31, 0.02, 0.10, CTKernel, 3)
print(CT_31_STD3)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([308.94983673095703, 305.13645911216736, 308.09714221954346, 304.50798749923706, 300.8761193752289], [15.942517380745864, 13.962824049367947, 15.099436882429538, 16.279029391012536, 15.95089547166273], [{4: 0.7595982225285884, 2: 0.7009564371791873, 1: 0.6299068181538424, 0.5: 0.5591280050222184, 0.25: 0.518630214552124}, {4: 0.8035415057341494, 2: 0.7527363479707647, 1: 0.6785786440073682, 0.5: 0.5822092815972428, 0.25: 0.5303939628023056}, {4: 0.7889867893791841, 2: 0.7400801436402336, 1: 0.6600554108847906, 0.5: 0.588413655334784, 0.25: 0.5378779771459827}, {4: 0.7788668098818474, 2: 0.7180719656283566, 1: 0.6423469387755102, 0.5: 0.5739124597207304, 0.25: 0.5309479054779807}, {4: 0.7432244570000127, 2: 0.698585585470685, 1: 0.644368487507493, 0.5: 0.5880087237108932, 0.25: 0.5376305686992233}], 15.43930770447941, {4: 0.77530631011258

In [117]:
CT_31_STD5 = testImages(allName[0:5], 31, 0.02, 0.10, CTKernel, 5)
print(CT_31_STD5)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([298.53315138816833, 299.9870641231537, 307.06282901763916, 304.38295555114746, 301.19123363494873], [13.789519578740277, 11.08398469990058, 12.944157274689157, 13.707791012904327, 14.37189455017503], [{4: 0.8174522089144376, 2: 0.7456024815667351, 1: 0.6616895825896429, 0.5: 0.5798200371742636, 0.25: 0.5323920776966727}, {4: 0.8647097272565215, 2: 0.8021629330322657, 1: 0.7134113732247905, 0.5: 0.6023293124962862, 0.25: 0.5416839978608354}, {4: 0.8432561623800521, 2: 0.7828511635096731, 1: 0.6875824920629259, 0.5: 0.6053341894671755, 0.25: 0.5461182653777096}, {4: 0.8341299677765843, 2: 0.7564446831364124, 1: 0.6618152524167562, 0.5: 0.5795515574650913, 0.25: 0.5285714285714286}, {4: 0.7845472980728762, 2: 0.7304449857793309, 1: 0.666713431198745, 0.5: 0.6034027574068642, 0.25: 0.546851684160853}], 13.195983641509917, {4: 0.829403285533

In [151]:
def blurImgs(imgPaths):
  imgLs = []
  imgRs = []
  ground_truths = []
  for imgPath in imgPaths:
      imgL, imgR, gt = readImgPair(imgPath)
      imgLBlur = GaussianFilter(imgL)
      imgRBlur = GaussianFilter(imgR)
      imgLs.append(imgLBlur)
      imgRs.append(imgRBlur)
      ground_truths.append(gt)
  return imgLs, imgRs, ground_truths

def testImagesBlur(imgLs, imgRs, groundTruths, windowSize, minSearch, maxSearch, costFunction):
    
  runTimes = []
  diffs = []
  rmsList = []
  thresholds = []
  thresholds_rates = []
  count_total = 0
  diff_sum = 0
  for i in range(len(imgLs)):
      print(f'image {i} starts.')
      startTime = time.time()
      disparity = fixedWindow(imgLs[i], imgRs[i], windowSize, minSearch, maxSearch, costFunction)
      duration = time.time()-startTime
      runTimes.append(duration)

      rms, threshold, diff, ground_truth_count = getErrorRate(disparity, groundTruths[i], windowSize)
      count_total += ground_truth_count
      rmsList.append(rms)
      thresholds.append(threshold)
      thresholds_rate = {}
      for key in threshold:
        thresholds_rate[key] = threshold[key] / ground_truth_count
      thresholds_rates.append(thresholds_rate)
      diffs.append(diff)
      print(f'image {i} done.')

  # overall threshold
  threshold_all = {4:0, 2:0, 1:0, 0.5:0, 0.25:0}
  for i in range(len(runTimes)):
    for key in threshold_all:
      threshold_all[key] += thresholds[i][key]
  for key in threshold_all:
    threshold_all[key] = threshold_all[key] / count_total

  # overall runtime
  runTimesSum = sum(runTimes)

  # overall rms
  for i in range(len(runTimes)):
    diff_sum += np.sum(square(diffs[i]))
  
  overall_rms = sqrt(diff_sum/count_total)
  
  return runTimes, rmsList, thresholds_rates, overall_rms, threshold_all

In [138]:
imgLs, imgRs, gts = blurImgs(allName[0:5])

In [139]:

SAD_preprocess = testImagesBlur(imgLs, imgRs, gts, 21, 0.02, 0.10, SAD)
print(SAD_preprocess)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([94.02587842941284, 93.3358702659607, 93.35524368286133, 94.18374562263489, 93.41206979751587], [16.151247583691358, 10.468043162413428, 16.352934752592937, 13.996945390897526, 14.793117745941597], [{4: 0.9328944218338521, 2: 0.8478692841752454, 1: 0.7440507541297582, 0.5: 0.6776155135264544, 0.25: 0.6506823078764664}, {4: 0.9038775865066945, 2: 0.7979481829247087, 1: 0.6995305164319249, 0.5: 0.6218628644293746, 0.25: 0.5797716339187388}, {4: 0.9031740308680026, 2: 0.8388676304679049, 1: 0.7779225159632003, 0.5: 0.7395992044569023, 0.25: 0.7155002965840496}, {4: 0.8836841623874668, 2: 0.8004782250793775, 1: 0.7139403917264449, 0.5: 0.6575986829210929, 0.25: 0.6292449008924255}, {4: 0.8429479223551606, 2: 0.7515514074368267, 1: 0.6503624087772427, 0.5: 0.587089807873703, 0.25: 0.5540013900610634}], 14.504497077648285, {4: 0.89396908127850

In [140]:
CT_preprocess = testImagesBlur(imgLs, imgRs, gts, 31, 0.02, 0.10, CT)
print(CT_preprocess)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([252.82368445396423, 252.9503197669983, 253.55032420158386, 252.04804348945618, 252.23154592514038], [10.107335096307583, 6.572082674322219, 9.420080332613205, 10.224406242588211, 11.375676768798236], [{4: 0.8408030625684708, 2: 0.7327515109737934, 1: 0.6302760989180074, 0.5: 0.5547335639286549, 0.25: 0.5143834857642265}, {4: 0.8808247667716442, 2: 0.7632063699566225, 1: 0.6489868679065898, 0.5: 0.5495394854120863, 0.25: 0.5006714599797968}, {4: 0.8578580006896633, 2: 0.7583324415272477, 1: 0.6530874326686406, 0.5: 0.5874029417710079, 0.25: 0.5449172998489875}, {4: 0.8448845327604726, 2: 0.7334317937701397, 1: 0.6200053705692803, 0.5: 0.5447905477980666, 0.25: 0.5070488721804511}, {4: 0.7699695180277271, 2: 0.6638948053107503, 1: 0.570829135153749, 0.5: 0.5083984848291606, 0.25: 0.46831277819582434}], 9.631836422808504, {4: 0.83968983771

In [141]:
def testImagesCTAD(imgLs, imgRs, groundTruths, windowSize1, windowSize2, minSearch, maxSearch, lambda_CT, lambda_AD):
    
  runTimes = []
  diffs = []
  rmsList = []
  thresholds = []
  thresholds_rates = []
  count_total = 0
  diff_sum = 0
  for i in range(len(imgLs)):
      print(f'image {i} starts.')
      startTime = time.time()
      disparity = multiWindowCTAD(imgLs[i], imgRs[i], windowSize1, windowSize2, lambda_CT, lambda_AD, minSearch, maxSearch)
      duration = time.time()-startTime
      runTimes.append(duration)

      rms, threshold, diff, ground_truth_count = getErrorRate(disparity, groundTruths[i], windowSize2)
      count_total += ground_truth_count
      rmsList.append(rms)
      thresholds.append(threshold)
      thresholds_rate = {}
      for key in threshold:
        thresholds_rate[key] = threshold[key] / ground_truth_count
      thresholds_rates.append(thresholds_rate)
      diffs.append(diff)
      print(f'image {i} done.')

  # overall threshold
  threshold_all = {4:0, 2:0, 1:0, 0.5:0, 0.25:0}
  for i in range(len(runTimes)):
    for key in threshold_all:
      threshold_all[key] += thresholds[i][key]
  for key in threshold_all:
    threshold_all[key] = threshold_all[key] / count_total

  # overall runtime
  runTimesSum = sum(runTimes)

  # overall rms
  for i in range(len(runTimes)):
    diff_sum += np.sum(square(diffs[i]))
  
  overall_rms = sqrt(diff_sum/count_total)
  
  return runTimes, rmsList, thresholds_rates, overall_rms, threshold_all

In [144]:
CTAD_20_6 = testImagesCTAD(imgLs, imgRs, gts, 31, 21, 0.02, 0.10, 20, 6)
print(CTAD_20_6)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([369.09594893455505, 375.15371108055115, 375.16450572013855, 380.5738670825958, 376.50210881233215], [11.833675449996653, 7.448729138957011, 11.638291059921979, 11.470245397265883, 12.301440227913858], [{4: 0.8663155374670817, 2: 0.7643045247785492, 1: 0.6661599233899929, 0.5: 0.592554464926981, 0.25: 0.553255925305243}, {4: 0.8834753376224425, 2: 0.7679707876890975, 1: 0.6575667999768157, 0.5: 0.5602272068625747, 0.25: 0.5125949110299658}, {4: 0.8666185929122228, 2: 0.7741192616801778, 1: 0.6734784075181149, 0.5: 0.609683759987904, 0.25: 0.5685574384442713}, {4: 0.8562580847477559, 2: 0.7543020657755478, 1: 0.6476683260815596, 0.5: 0.574850064677982, 0.25: 0.5378072204147231}, {4: 0.7935014645286204, 2: 0.6923000546095418, 1: 0.6016109814824009, 0.5: 0.5409075112942462, 0.25: 0.5020478578166112}], 11.047558209854747, {4: 0.8538905129465

In [145]:
CTAD_6_20 = testImagesCTAD(imgLs, imgRs, gts, 31, 21, 0.02, 0.10, 6, 20)
print(CTAD_6_20)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([375.3957107067108, 359.8263280391693, 360.09203934669495, 365.1010835170746, 389.5898811817169], [10.84654440417458, 7.570437477747062, 10.31030676601492, 10.975832859559599, 11.73346559352838], [{4: 0.8455949245870242, 2: 0.7395499162078046, 1: 0.6384725879818052, 0.5: 0.5638017716064161, 0.25: 0.5237491022264783}, {4: 0.8819683533298557, 2: 0.7653509534573697, 1: 0.6524894221294847, 0.5: 0.5542340462528256, 0.25: 0.5062308004405031}, {4: 0.8605589737028809, 2: 0.7622442689493946, 1: 0.6583002826271531, 0.5: 0.5932960374044824, 0.25: 0.5515532862675769}, {4: 0.8493068349600825, 2: 0.7405432950491945, 1: 0.62936249722342, 0.5: 0.5561261155318621, 0.25: 0.5191355363046006}, {4: 0.7753934369259793, 2: 0.6705307054559897, 1: 0.5788859653477635, 0.5: 0.5174750533684158, 0.25: 0.4782306508464479}], 10.349398694264776, {4: 0.843299288670805, 

In [146]:
CTAD_3_20 = testImagesCTAD(imgLs, imgRs, gts, 31, 21, 0.02, 0.10, 3, 20)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([375.05690145492554, 359.35785937309265, 359.83822417259216, 356.5094254016876, 355.57680678367615], [10.85119629656357, 7.570317872291448, 10.31575247790673, 10.977139736327745, 11.750410138833487], [{4: 0.8450203495331577, 2: 0.7389035192722049, 1: 0.6374431410102944, 0.5: 0.5625927699305722, 0.25: 0.5225401005506344}, {4: 0.8819335767692575, 2: 0.7653045847099055, 1: 0.6522923549527618, 0.5: 0.5540022025155046, 0.25: 0.5060916941981105}, {4: 0.8601053745682086, 2: 0.7615696856209074, 1: 0.65752102257528, 0.5: 0.5924818851114807, 0.25: 0.5506926109864037}, {4: 0.8489671122260986, 2: 0.7396286569192374, 1: 0.6285001241294605, 0.5: 0.5552506761789032, 0.25: 0.5182731632106412}, {4: 0.7751328004765924, 2: 0.670456237899022, 1: 0.5788239090502905, 0.5: 0.5173509407734697, 0.25: 0.47806930447301793}], 10.35550369889209, {4: 0.84296748036397

In [147]:
print(CTAD_3_20)

([375.05690145492554, 359.35785937309265, 359.83822417259216, 356.5094254016876, 355.57680678367615], [10.85119629656357, 7.570317872291448, 10.31575247790673, 10.977139736327745, 11.750410138833487], [{4: 0.8450203495331577, 2: 0.7389035192722049, 1: 0.6374431410102944, 0.5: 0.5625927699305722, 0.25: 0.5225401005506344}, {4: 0.8819335767692575, 2: 0.7653045847099055, 1: 0.6522923549527618, 0.5: 0.5540022025155046, 0.25: 0.5060916941981105}, {4: 0.8601053745682086, 2: 0.7615696856209074, 1: 0.65752102257528, 0.5: 0.5924818851114807, 0.25: 0.5506926109864037}, {4: 0.8489671122260986, 2: 0.7396286569192374, 1: 0.6285001241294605, 0.5: 0.5552506761789032, 0.25: 0.5182731632106412}, {4: 0.7751328004765924, 2: 0.670456237899022, 1: 0.5788239090502905, 0.5: 0.5173509407734697, 0.25: 0.47806930447301793}], 10.35550369889209, {4: 0.842967480363972, 2: 0.7359169171375357, 1: 0.6316298084957458, 0.5: 0.5568324658685506, 0.25: 0.5154969011041709})


In [148]:
CTAD_3_40 = testImagesCTAD(imgLs, imgRs, gts, 31, 21, 0.02, 0.10, 3, 40)
print(CTAD_3_40)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([370.9216637611389, 385.6996419429779, 372.52010679244995, 376.493360042572, 383.4269349575043], [10.85120157091734, 7.574303192867102, 10.317276763556617, 10.977131962077225, 11.75512204034563], [{4: 0.8450203495331577, 2: 0.7389035192722049, 1: 0.6374431410102944, 0.5: 0.5625927699305722, 0.25: 0.5225401005506344}, {4: 0.8819335767692575, 2: 0.7653045847099055, 1: 0.6522923549527618, 0.5: 0.5540022025155046, 0.25: 0.5060916941981105}, {4: 0.8601053745682086, 2: 0.7615580548738645, 1: 0.65752102257528, 0.5: 0.5924818851114807, 0.25: 0.5506926109864037}, {4: 0.8489671122260986, 2: 0.7396286569192374, 1: 0.6284870578704611, 0.5: 0.5552376099199038, 0.25: 0.5182600969516418}, {4: 0.7751824455145708, 2: 0.6705058829370004, 1: 0.5788735540882689, 0.5: 0.5174005858114481, 0.25: 0.47811894951099637}], 10.357471743433605, {4: 0.842977168197748,

In [149]:
CTAD_2_20 = testImagesCTAD(imgLs, imgRs, gts, 31, 21, 0.02, 0.10, 2, 20)
print(CTAD_2_20)

image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
([372.75204730033875, 372.7479569911957, 374.6696209907532, 372.4165201187134, 373.26558923721313], [10.850905927367423, 7.570317872291448, 10.31572090857466, 10.97715881855568, 11.755189051549605], [{4: 0.8450083792195355, 2: 0.7389035192722049, 1: 0.63741920038305, 0.5: 0.56258079961695, 0.25: 0.5225401005506344}, {4: 0.8819335767692575, 2: 0.7653045847099055, 1: 0.6522923549527618, 0.5: 0.5540022025155046, 0.25: 0.5060916941981105}, {4: 0.8600821130741227, 2: 0.7615580548738645, 1: 0.65752102257528, 0.5: 0.5924818851114807, 0.25: 0.5506809802393607}, {4: 0.8489540459670991, 2: 0.7395894581422393, 1: 0.6284609253524623, 0.5: 0.5551984111429057, 0.25: 0.5182078319156442}, {4: 0.7750335104006355, 2: 0.6703693590825597, 1: 0.5788239090502905, 0.5: 0.5173509407734697, 0.25: 0.47806930447301793}], 10.356497690473189, {4: 0.8429384168626435, 

In [152]:
imgLs, imgRs, gts = blurImgs(allName)
CT_preprocess_all = testImagesBlur(imgLs, imgRs, gts, 31, 0.02, 0.10, CT)


image 0 starts.
image 0 done.
image 1 starts.
image 1 done.
image 2 starts.
image 2 done.
image 3 starts.
image 3 done.
image 4 starts.
image 4 done.
image 5 starts.
image 5 done.
image 6 starts.
image 6 done.
image 7 starts.
image 7 done.
image 8 starts.
image 8 done.
image 9 starts.
image 9 done.
image 10 starts.
image 10 done.
image 11 starts.
image 11 done.
image 12 starts.
image 12 done.
image 13 starts.
image 13 done.
image 14 starts.
image 14 done.
image 15 starts.
image 15 done.
image 16 starts.
image 16 done.
image 17 starts.
image 17 done.
image 18 starts.
image 18 done.
image 19 starts.
image 19 done.
image 20 starts.
image 20 done.
image 21 starts.
image 21 done.
image 22 starts.
image 22 done.
image 23 starts.
image 23 done.
image 24 starts.
image 24 done.
([252.82368445396423, 252.9503197669983, 253.55032420158386, 252.04804348945618, 252.23154592514038], [10.107335096307583, 6.572082674322219, 9.420080332613205, 10.224406242588211, 11.375676768798236], [{4: 0.84080306256

In [153]:
print(CT_preprocess_all)

([250.92716002464294, 255.28648233413696, 255.8661105632782, 256.16523122787476, 255.05429530143738, 257.00569105148315, 259.00325894355774, 261.22480368614197, 261.62008023262024, 261.05768609046936, 257.2607753276825, 259.60646748542786, 257.73227763175964, 258.271870136261, 261.6449599266052, 260.76598477363586, 261.60365080833435, 261.4193341732025, 261.8423421382904, 264.7058403491974, 262.25862526893616, 262.0320534706116, 262.66295170783997, 261.80388259887695, 262.80220675468445], [10.107335096307583, 6.572082674322219, 9.420080332613205, 10.224406242588211, 11.375676768798236, 10.234045351792913, 10.428404602031376, 10.385893318967597, 9.057854876066207, 10.954648485602034, 16.840311895960998, 9.443802668688551, 13.999671911159636, 10.46573122379419, 9.932384107085628, 9.514842846622455, 8.046485790551978, 7.4257942197817455, 10.399010563413102, 11.883439700735678, 5.938505850295663, 9.577713387089672, 16.045850762502752, 10.598536856980697, 11.664439877117934], [{4: 0.8408030

In [156]:
print(sum(CT_preprocess_all[0]) / 60)

108.16040036678314
